In [ ]:
# new
!pip install pandasql
!pip install numpy_financial

In [40]:
import numpy as np
import pandas as pd
import pandasql as ps
import numpy_financial as npf

In [41]:
df = pd.DataFrame(dict(index=['demand','price','sales','cost','expenses','cash'],year1=[0,10,0,0,0,0],year2=[0,10,0,0,0,0],year3=[0,10,0,0,0,0],year4=[0,10,0,0,0,0]))

In [42]:
df.set_index('index',inplace=True)

In [43]:
df

,year1,year2,year3,year4
index,,,,
demand,0,0,0,0
price,10,10,10,10
sales,0,0,0,0
cost,0,0,0,0
expenses,0,0,0,0
cash,0,0,0,0


In [44]:
# not relevent
# calculate all at once
# import numpy as np
# import pandas as pd

# class make_fr():

#   def __init__(self,Number_of_Periods,Number_of_Simulations):
#     self.pd = Number_of_Periods
#     self.Number_of_Simulations = Number_of_Simulationscas
#     return(None)

#   def NDist(self,**details):
#     """
#     Provide variable name , mean & std
#     e.g.:
#       demand = [50,5]
#       temp = [20,2]

#     """
#     ND = {}

#     for d in details:
#       col = [np.random.normal(details[d][0],details[d][1],self.pd) for i in range(self.Number_of_Simulations)]
#       ND.update({d:col})

#     return(ND)

In [45]:

# one by one
import numpy as np
import pandas as pd
import pandasql as ps

class make_fr():

  def __init__(self,data,number_of_simulations,feature_to_simulate,sql_query,calculate_NPV_IRR=False,Required_Rate=None,Initial_investment=None):
    self.data = data.transpose()
    self.pd = data.shape[1]
    self.number_of_simulations = number_of_simulations
    self.query = sql_query
    self.feature_to_simulate = feature_to_simulate
    self.all_random_numbers = {}
    self.calculate_NPV_IRR = calculate_NPV_IRR
    self.Required_Rate = Required_Rate

    if Initial_investment < 0:
      self.Initial_investment = Initial_investment
    else:
      self.Initial_investment = Initial_investment * -1.0

    return(None)

  # normal dist
  def NDist(self,**details):
    """
    Provide variable name , mean & std
    e.g.:
      demand = [50,5]
      temperature = [20,2]

    """
    self.n_details = details
    self.ND = {}

    for d in details:
      col = np.random.normal(details[d][0],details[d][1],(self.number_of_simulations,self.pd)) # self.pd is numbers of years (columns)
      self.ND.update({d:col})

    # update the global dict
    self.all_random_numbers.update(self.ND)
    return(self.ND)


  #triangle distribution
  def TriDist(self,**details):
    """
    Provide variable name , lowest value, most likely value & maximum value
    e.g.:
      demand = [10,100,250]
      temp = [-10,18,40]

    """
    self.t_details = details
    self.TD = {}
    for d in details:
      col = np.random.triangular(details[d][0],details[d][1],details[d][2],(self.number_of_simulations,self.pd))
      self.TD.update({d:col})
    
    # update the global dict
    self.all_random_numbers.update(self.TD)
    return(self.TD)


  def simul8(self):

    copy_data = self.data.copy()
 
    self.full_data = []
    self.feature_only_sum = []
    self.NPV = []
    self.IRR =[]

   
    for i in range(self.number_of_simulations):

      # get the random number for the current i
      for k,v in zip(fr.all_random_numbers.keys(),fr.all_random_numbers.values()):
        #print(k,v[i])
        copy_data[k] = v[i]
        copy_data_updated = copy_data.copy()

      # once we have all the random numbers in place, we need to apply the sql formulas
      data_index  = self.data.index
      df = copy_data_updated.reset_index(drop=True)
      df_refreshed = ps.sqldf(self.query,locals())

      # now check if we need to calculate NPV/IRR or a simple sum 
      if (self.Required_Rate is not None) & (self.calculate_NPV_IRR == True) & (self.Initial_investment is not None):
        # add initial investment to the cash
        c = list(df_refreshed[self.feature_to_simulate])
        c.insert(0,self.Initial_investment)

        #calculate npv
        npv = npf.npv(self.Required_Rate,c)
        self.NPV.append(npv)

        #calculate irr
        irr = npf.mirr(c,self.Required_Rate,self.Required_Rate)
        self.IRR.append(irr)
      
      # now calculate the sum of all the cash
      if self.Initial_investment is None:
        cash = np.sum(df_refreshed[self.feature_to_simulate])
      else:
        cash = self.Initial_investment + np.sum(df_refreshed[self.feature_to_simulate])
      # add to list
      self.feature_only_sum.append(cash)

      # now save the full data frame
      self.full_data.append(df_refreshed)


    return(self.full_data)







In [46]:
q = "select demand,price,demand*price as sales, cost ,demand*cost as expenses, (demand*price)-(demand*cost) as cash from df"

In [47]:
fr = make_fr(data=df,
             number_of_simulations= 10000,
             feature_to_simulate= 'cash',
             sql_query= q,
             calculate_NPV_IRR= True,
             Required_Rate= 0.20,
             Initial_investment= -500)

In [48]:
fr.NDist(price=[10,2])

{'price': array([[11.38162798, 12.29924881, 11.23266344, 10.3714007 ],
        [10.13881802, 12.85756424, 11.617153  , 12.24892373],
        [11.21090509,  8.40502572,  9.30513956,  9.08047069],
        ...,
        [ 7.93288327, 10.67104278,  7.89450942, 13.1180334 ],
        [ 7.75807222, 11.08615579,  5.46245568,  9.51523877],
        [11.700255  ,  9.02598075, 10.83191962, 11.4412561 ]])}

In [49]:
fr.TriDist(cost=[2,8,12],demand=[10,100,250])

{'cost': array([[ 5.83837878,  8.32730133,  8.02898835,  4.57449269],
        [ 9.06815882,  5.17440996,  6.92089795,  6.23781022],
        [ 3.24551574,  3.61724906,  6.46482137,  7.89968435],
        ...,
        [11.64349092,  3.94368805,  4.85757244,  5.33506621],
        [ 7.19494059,  7.62939717,  9.62218754,  7.04586912],
        [ 6.7398787 , 10.13221428, 11.16643106,  9.68118578]]),
 'demand': array([[146.55109707, 191.7471161 , 238.92747324, 136.14868347],
        [ 83.26334911, 195.36782497, 136.19462101, 126.41160483],
        [110.37295281, 141.34177489,  44.20110613,  88.76705275],
        ...,
        [143.67390022,  83.17059005,  51.53528052,  86.62551091],
        [129.62812699,  14.64335094, 133.60745976, 184.77768354],
        [ 66.30396644,  46.68135176, 219.74244766, 104.36162447]])}

In [50]:
fr.Initial_investment

-500

In [51]:
 %%time
 l = fr.simul8()

CPU times: user 59.6 s, sys: 176 ms, total: 59.8 s
Wall time: 59.9 s


In [52]:
fr.full_data[3]

,demand,price,sales,cost,expenses,cash
0,23.665538,9.812898,232.227498,9.647852,228.321612,3.905886
1,113.584086,8.444051,959.109858,6.499429,738.231682,220.878175
2,113.473704,11.380361,1291.371675,11.215364,1272.648887,18.722788
3,56.367959,8.319840,468.972424,8.806255,496.390644,-27.418220


In [53]:
fr.feature_only_sum[:10]

[2628.666113593318,
 2489.6669931012766,
 1286.236522015414,
 -283.911371111712,
 946.334906604488,
 1793.157323264762,
 442.4372329418767,
 -182.57505825743766,
 1422.3346426650746,
 444.25707057139095]

In [24]:
fr.NPV[3]

-298.3736601162641

In [26]:
fr.IRR[3]*100

nan

In [27]:
c = list(fr.full_data[3]['cash'])

In [28]:
c.insert(0,-500)

In [55]:
fr.IRR

[0.7032694384045042,
 0.6650356464115481,
 0.5312860605774263,
 -0.09302791666291466,
 0.4084473955101757,
 0.5558303641246558,
 0.16603593994506882,
 -0.010936619672864079,
 0.5284787006773306,
 0.2573150810573266,
 -0.07229497675619923,
 0.26112810625580085,
 0.2375753704793564,
 0.46572125228823436,
 0.5454439436574947,
 0.7320733724754327,
 -0.7928095156678493,
 0.2614404045797223,
 0.38056940080333135,
 -0.13534540828260633,
 0.31505979336093826,
 0.47105266243402144,
 0.4867281612686336,
 0.3835857901944797,
 0.4868898440282654,
 0.1388954390615753,
 0.4305865378363938,
 0.4103966152271452,
 0.3124440147097194,
 0.1302993956718499,
 -0.3037021709194495,
 0.22175123827538168,
 0.5421134530170568,
 0.39821423605588135,
 0.22643637141263384,
 0.1773121783429883,
 0.3959533898687315,
 0.2958127261277468,
 0.6420704376255524,
 0.1265595015816381,
 0.1555779569900826,
 0.2985221019753941,
 0.07391495650397673,
 0.49196886983144483,
 0.5066938029987069,
 0.3906346310848241,
 -0.06465972

In [31]:
c

[-500,
 383.11629838763247,
 -134.06577003024586,
 -30.617168259389018,
 -14.137274476093353]

In [29]:
(npf.irr(c)) *100

nan

In [54]:
np.min([i for i in fr.IRR if i is not np.nan])

-0.7928095156678493

In [21]:
npf.npv(.20,c)

100.17661913075372

In [165]:
import plotly.express as px
fig = px.histogram( pd.DataFrame(fr.feature_only_sum,columns=['cash']), x="cash",histnorm='probability')
fig.show()

In [39]:
import plotly.express as px
fig = px.histogram( pd.DataFrame(fr.feature_only_sum,columns=['cash']), x="cash",histnorm='probability',cumulative=True)
fig.show()

In [163]:
import plotly.express as px
fig = px.histogram( pd.DataFrame(stats.zscore(fr.feature_only_sum),columns=['cash']), x="cash")
fig.show()

In [59]:
import plotly.express as px
fig = px.histogram( pd.DataFrame(fr.IRR,columns=['cash']), x="cash",histnorm='probability')
fig.show()

In [106]:
import plotly.express as px
fig = px.histogram( pd.DataFrame(fr.NPV,columns=['NPV']), x="NPV")
fig.show()

In [137]:
pd.DataFrame(fr.feature_only_sum,columns=['cash'])

,cash
0,610.160528
1,2077.177002
2,2050.438713
3,947.435656
4,1262.433286
...,...
995,2060.232245
996,443.686498
997,2364.171075
998,972.717505


In [167]:
np.average(fr.IRR)

nan

In [153]:
from scipy import stats

In [153]:
stats.zscore()